In [14]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.manifold import TSNE
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from geomloss import SamplesLoss
from torch.autograd import Function
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.functional import normalize
#from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryF1Score
torch.manual_seed(0)

In [15]:
class TarNet(nn.Module):
    def __init__(self,params):
        super(TarNet, self).__init__()
        self.encoder1 = nn.Linear(25, params['RL11'])
        self.encoder2 = nn.Linear(params['RL11'], params['RL21'])
        self.encoder3 = nn.Linear(params['RL21'], params['RL32'])

        self.regressor1_y0 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG012']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y0 = nn.Sequential(
            nn.Linear(params['RG012'], params['RG022']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y0 = nn.Linear(params['RG022'], 1)

        self.regressor1_y1 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG112']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y1 = nn.Sequential(
            nn.Linear(params['RG112'], params['RG122']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y1 = nn.Linear(params['RG122'], 1)


    def forward(self, inputs):
        x = nn.functional.elu(self.encoder1(inputs))
        x = nn.functional.elu(self.encoder2(x))
        phi = nn.functional.elu(self.encoder3(x))

        out_y0 = self.regressor1_y0(phi)
        out_y0 = self.regressor2_y0(out_y0)
        y0 = self.regressorO_y0(out_y0)

        out_y1 = self.regressor1_y1(phi)
        out_y1 = self.regressor2_y1(out_y1)
        y1 = self.regressorO_y1(out_y1)

        concat = torch.cat((y0, y1), 1)
        return concat

In [16]:
def objective(trial,i):

    params = {
          'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
          'batch_size':trial.suggest_int('batch_size', 8, 256),
          'RL11':trial.suggest_int('RL11', 16, 512),
          'RL21': trial.suggest_int('RL21', 16, 512),
          'RL32': trial.suggest_int('RL32', 16, 512),
          'RG012':trial.suggest_int('RG012', 16, 512),
        'RG022':trial.suggest_int('RG022', 16, 512),
        'RG112':trial.suggest_int('RG112', 16, 512),
        'RG122':trial.suggest_int('RG122', 16, 512),
          
          }

    model = TarNet(params)

    pehe,model= train_evaluate(params, model, trial,i)

    return pehe

In [17]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [18]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
    #x_data=data.iloc[:, 5:30]
    y_data=data.iloc[:, 1]
    return x_data,y_data

In [19]:
def get_dataloader(x_data,y_data,batch_size):

    x_train_sr=x_data[x_data['treatment']==0]
    y_train_sr=y_data[x_data['treatment']==0]
    x_train_tr=x_data[x_data['treatment']==1]
    y_train_tr=y_data[x_data['treatment']==1]


    train_data_sr = Data(np.array(x_train_sr), np.array(y_train_sr))
    train_dataloader_sr = DataLoader(dataset=train_data_sr, batch_size=batch_size)

    train_data_tr = Data(np.array(x_train_tr), np.array(y_train_tr))
    train_dataloader_tr = DataLoader(dataset=train_data_tr, batch_size=batch_size)


    return train_dataloader_sr, train_dataloader_tr

In [20]:
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = torch.sum((1. - t_true) * torch.square(y_true - y0_pred))
    loss1 = torch.sum(t_true * torch.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

In [21]:
def cal_pehe(data,y,model):
    #data,y=get_data('test',i)

    data=data.to_numpy()
    data=torch.from_numpy(data.astype(np.float32))



    concat_pred=model(data[:,5:30])
    #dont forget to rescale the outcome before estimation!
    #y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    #y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    cate_pred=concat_pred[:,1]-concat_pred[:,0]
    cate_true=data[:,4]-data[:,3] #Hill's noiseless true values


    cate_err=torch.mean( torch.square( ( (cate_true) - (cate_pred) ) ) )

    return torch.sqrt(cate_err).item()


In [22]:
def loss_cal(X_data,y_data,net):
    
    x_train_sr=X_data[X_data['treatment']==0]
    y_train_sr=y_data[X_data['treatment']==0]
    x_train_tr=X_data[X_data['treatment']==1]
    y_train_tr=y_data[X_data['treatment']==1]
    xs_t=x_train_sr.iloc[:,0].to_numpy()
    xt_t=x_train_tr.iloc[:,0].to_numpy()
    
    xs=x_train_sr.iloc[:,5:30].to_numpy()
    xt=x_train_tr.iloc[:,5:30].to_numpy()
    xs_t=torch.from_numpy(xs_t.astype(np.float32))
    xt_t=torch.from_numpy(xt_t.astype(np.float32))
    y_train_sr=y_train_sr.to_numpy()
    y_train_tr=y_train_tr.to_numpy()
    xs=torch.from_numpy(xs.astype(np.float32))
    xt=torch.from_numpy(xt.astype(np.float32))
    
    y_train_sr=torch.from_numpy(y_train_sr.astype(np.float32))
    y_train_tr=torch.from_numpy(y_train_tr.astype(np.float32))
    
    
    input_data=torch.cat((xs,xt),0)
    true_y=torch.unsqueeze(torch.cat((y_train_sr,y_train_tr),0), dim=1)
    true_t=torch.unsqueeze(torch.cat((xs_t,xt_t),0), dim=1)
    
    
    concat_true=torch.cat((true_y,true_t),1)
    concat_pred=net(input_data)
    loss=regression_loss(concat_true, concat_pred)
    
    return loss.item()

    

In [23]:

#criterion_reg=nn.MSELoss()
#criterion_reg=regression_loss(concat_true,concat_pred)
epochs=300
#batch_size=32

In [24]:
train_loss=[]
val_loss=[]
pehe_error=[]
num_files=2
def train_evaluate(param, model, trial,file_num):
    #for nf in range(1,num_files):
    x_data,y_data=get_data('train',file_num)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.20)
    
    #net=TarNet(25,.01)
    #opt_net = torch.optim.Adam(net.parameters(), lr=1e-4)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])
    
    if use_cuda:

        #model = model.cuda()
        model = model
        #criterion = criterion.cuda()

    for ep in range(1,epochs+1 ):

        train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,param['batch_size'])

        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):

            xs,ys=train_source_data
            xt,yt=train_target_data

            xs_train=xs[:,5:30]
            xt_train=xt[:,5:30]

            train_x=torch.cat((xs_train,xt_train),0)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1)
            concat_true=torch.cat((train_y,true_t),1)
            concat_pred=model(train_x)

            model.zero_grad()

            #source_mse=criterion_reg(y0,ys)
            #target_mse=criterion_reg(y1,yt)

            #combined loss
            combined_loss=regression_loss(concat_true,concat_pred)
            #print('Training loss: ',combined_loss.item())
            # backward propagation
            combined_loss.backward()

            # optimize
            optimizer.step()
        #train_loss.append(loss_cal(X_train,y_train,net))
        #val_loss.append(loss_cal(X_val,y_val,net))
        
        # Add prune mechanism
        #trial.report(accuracy, ep)

        #if trial.should_prune():
        #   raise optuna.exceptions.TrialPruned()
            
    #return cal_pehe(X_val,y_val,model),model
    return loss_cal(X_val,y_val,model),model

        
        

In [25]:
pehe_total=[]
for i in range(1,101):
    func = lambda trial: objective(trial, i)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(func, n_trials=50)
    best_trial = study.best_trial
    best_model=TarNet(study.best_trial.params)
    best_val,model=train_evaluate(study.best_trial.params, best_model, study.best_trial,i)
    data,y=get_data('test',i)
    pehe=cal_pehe(data,y,model)

    pehe_total.append(pehe)


[I 2023-05-04 23:46:00,993] A new study created in memory with name: no-name-d8c41de0-6be9-46f4-9c03-6a5243426522
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-04 23:46:02,457] Trial 0 finished with value: 205.7620086669922 and parameters: {'learning_rate': 0.00031489116479568613, 'optimizer': 'Adam', 'batch_size': 157, 'RL11': 93, 'RL21': 93, 'RL32': 44, 'RG012': 446, 'RG022': 314, 'RG112': 367, 'RG122': 26}. Best is trial 0 with value: 205.7620086669922.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_

[I 2023-05-04 23:46:36,440] Trial 12 finished with value: 294.2793884277344 and parameters: {'learning_rate': 1.1214711151718291e-05, 'optimizer': 'SGD', 'batch_size': 132, 'RL11': 212, 'RL21': 492, 'RL32': 126, 'RG012': 348, 'RG022': 397, 'RG112': 189, 'RG122': 160}. Best is trial 0 with value: 205.7620086669922.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-04 23:46:37,935] Trial 13 failed with parameters: {'learning_rate': 0.0007999912943355327, 'optimizer': 'SGD', 'batch_size': 124, 'RL11': 224, 'RL21': 388, 'RL32': 130, 'RG012': 303, 'RG022': 375, 'RG112': 148, 'RG122': 151} because of the following error: The value nan is not acceptable..
[W 2023-05-04 23:46:37,936] Trial 13 failed with v

/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-04 23:46:56,397] Trial 24 failed with parameters: {'learning_rate': 0.0006025951472529953, 'optimizer': 'SGD', 'batch_size': 147, 'RL11': 492, 'RL21': 390, 'RL32': 112, 'RG012': 337, 'RG022': 388, 'RG112': 147, 'RG122': 140} because of the following error: The value nan is not acceptable..
[W 2023-05-04 23:46:56,398] Trial 24 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('lea

[W 2023-05-04 23:47:14,952] Trial 35 failed with parameters: {'learning_rate': 0.00077405027888692, 'optimizer': 'SGD', 'batch_size': 131, 'RL11': 496, 'RL21': 407, 'RL32': 128, 'RG012': 175, 'RG022': 413, 'RG112': 155, 'RG122': 156} because of the following error: The value nan is not acceptable..
[W 2023-05-04 23:47:14,952] Trial 35 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-04 23:47:16,434] Trial 36 failed with parameters: {'learning_rate': 0.0007991244910074033, 'optimizer': 'SGD', 'batch_size': 130, 'RL11': 27, 'RL21': 395, 'RL32': 108, 'RG012': 321, 'RG022': 394, 'RG112': 151, 'RG122': 157} because of the following error: The value nan is not acceptable..
[W 2023

[W 2023-05-04 23:47:33,682] Trial 46 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-04 23:47:35,244] Trial 47 failed with parameters: {'learning_rate': 0.0006762593268436284, 'optimizer': 'SGD', 'batch_size': 133, 'RL11': 22, 'RL21': 395, 'RL32': 126, 'RG012': 338, 'RG022': 383, 'RG112': 140, 'RG122': 141} because of the following error: The value nan is not acceptable..
[W 2023-05-04 23:47:35,245] Trial 47 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float

...

/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-05 02:57:28,819] Trial 2 failed with parameters: {'learning_rate': 0.00014742753159914678, 'optimizer': 'SGD', 'batch_size': 203, 'RL11': 115, 'RL21': 271, 'RL32': 310, 'RG012': 39, 'RG022': 317, 'RG112': 100, 'RG122': 48} because of the following error: The value nan is not acceptable..
[W 2023-05-05 02:57:28,820] Trial 2 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learni

[W 2023-05-05 02:58:06,436] Trial 14 failed with parameters: {'learning_rate': 0.0013715119128958982, 'optimizer': 'SGD', 'batch_size': 196, 'RL11': 449, 'RL21': 503, 'RL32': 324, 'RG012': 123, 'RG022': 32, 'RG112': 496, 'RG122': 211} because of the following error: The value nan is not acceptable..
[W 2023-05-05 02:58:06,436] Trial 14 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-05 02:58:11,073] Trial 15 failed with parameters: {'learning_rate': 0.0018091728041976268, 'optimizer': 'SGD', 'batch_size': 14, 'RL11': 509, 'RL21': 508, 'RL32': 338, 'RG012': 130, 'RG022': 27, 'RG112': 499, 'RG122': 201} because of the following error: The value nan is not acceptable..
[W 2023

[W 2023-05-05 02:58:46,347] Trial 25 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-05 02:58:50,770] Trial 26 failed with parameters: {'learning_rate': 0.0023838100964866512, 'optimizer': 'SGD', 'batch_size': 13, 'RL11': 499, 'RL21': 499, 'RL32': 308, 'RG012': 121, 'RG022': 20, 'RG112': 484, 'RG122': 205} because of the following error: The value nan is not acceptable..
[W 2023-05-05 02:58:50,771] Trial 26 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float`

[W 2023-05-05 02:59:21,201] Trial 37 failed with parameters: {'learning_rate': 0.001626058338039452, 'optimizer': 'SGD', 'batch_size': 195, 'RL11': 505, 'RL21': 510, 'RL32': 346, 'RG012': 123, 'RG022': 483, 'RG112': 501, 'RG122': 204} because of the following error: The value nan is not acceptable..
[W 2023-05-05 02:59:21,202] Trial 37 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-05 02:59:23,823] Trial 38 failed with parameters: {'learning_rate': 0.001155414201413222, 'optimizer': 'SGD', 'batch_size': 200, 'RL11': 467, 'RL21': 493, 'RL32': 322, 'RG012': 115, 'RG022': 60, 'RG112': 501, 'RG122': 512} because of the following error: The value nan is not acceptable..
[W 2023

[W 2023-05-05 02:59:55,980] Trial 48 failed with value nan.
/tmp/ipykernel_1138773/1595419454.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[W 2023-05-05 02:59:58,239] Trial 49 failed with parameters: {'learning_rate': 0.0011621788894598084, 'optimizer': 'SGD', 'batch_size': 199, 'RL11': 491, 'RL21': 373, 'RL32': 341, 'RG012': 139, 'RG022': 511, 'RG112': 123, 'RG122': 201} because of the following error: The value nan is not acceptable..
[W 2023-05-05 02:59:58,240] Trial 49 failed with value nan.


In [26]:
print(np.mean(pehe_total))

nan


In [29]:
np.mean(pehe_total[0:99])

0.9699601031313039

In [27]:
pehe_total

[0.7243529558181763,
 1.0974527597427368,
 0.9394102096557617,
 0.5717839002609253,
 0.9096927046775818,
 1.1487631797790527,
 0.3492894768714905,
 0.7218011021614075,
 0.8568695187568665,
 1.5668202638626099,
 0.9571043252944946,
 0.8009145259857178,
 0.6833164691925049,
 1.4453110694885254,
 0.8261359930038452,
 0.8569546341896057,
 0.5389509201049805,
 0.44863349199295044,
 0.9471801519393921,
 1.5991759300231934,
 0.6929744482040405,
 0.6652252674102783,
 0.784331738948822,
 0.6690612435340881,
 0.6766694188117981,
 0.9668810367584229,
 0.9014957547187805,
 0.7697464823722839,
 1.324318528175354,
 0.621450662612915,
 0.37054741382598877,
 0.6356532573699951,
 0.7808274626731873,
 0.6976386308670044,
 0.6815195679664612,
 0.5998408198356628,
 0.4749812185764313,
 0.3917040228843689,
 1.1139411926269531,
 1.7644257545471191,
 1.743551254272461,
 0.5670009255409241,
 0.36650827527046204,
 0.9405661225318909,
 0.6831498742103577,
 0.5027104616165161,
 0.9520105719566345,
 1.12562572956

In [44]:
#for key, value in best_trial.params.items():
#    print("{}: {}".format(key, value))

[]

In [ ]:
#ate_pred=torch.mean(cate_pred)
#print("Estimated ATE (True is 4):", ate_pred.detach().numpy(),'\n\n')

#print("Individualized CATE Estimates: BLUE")
#print(pd.Series(cate_pred.detach().numpy()).plot.kde(color='blue'))
#print("Individualized CATE True: Green")
#print(pd.Series(cate_true.detach().numpy()).plot.kde(color='green'))

#print("\nError CATE Estimates: RED")
#print(pd.Series(cate_pred.detach().numpy()-cate_true.detach().numpy()).plot.kde(color='red'))